# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [11]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math


## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [12]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [13]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [14]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [15]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [16]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [17]:
gamma = 1
sigma = .005

In [18]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.5%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 2:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .1629.


Pregunta 3:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 382 y 554 días.


Pregunta 4:

Considere un swap a 1Y con cupones semestrales de 1.0% (lineal). Si la tasa
cupón cero a 0.5Y es 0.9% (exp), calcule la tasa cero a 1Y.


Pregunta 5:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,110.00 (365 días después).


Pregunta 6:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull

## Respuestas

### Pregunta 1

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.5%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.

In [19]:
gamma= 1.0
sigma=(0.5/100)

strike=(98.5/100)
nocional=1


In [20]:
def zrate(t: float) -> float:
    return curva(t)

In [22]:
#Función A de la fórmula para un bono cupón cero:

def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [23]:
#Función  𝐵 de la fórmula para un bono cupón cero:

def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [24]:
from scipy.stats import norm
T0=1
TB=2
    
#Se obtienen los valores que van en la ecuacion para obtener calls y puts
zero_T0=math.exp(-zrate(T0)*T0)
zero_TB=math.exp(-zrate(TB)*TB)

Sz=b_hw(gamma,T0,TB)*math.sqrt((sigma**2/(2*gamma)*(1-math.exp(-2*gamma*T0))))
d1=(math.log(zero_TB/(strike*zero_T0))+ 0.5*(Sz**2))/Sz
d2=d1- Sz
call= zero_T0*((zero_TB/zero_T0)*norm.cdf(d1)- strike*norm.cdf(d2))

In [25]:
print (f'El valor de la call es igual a:  {call:.8}')

El valor de la call es igual a:  0.014167931


### Pregunta 2

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .1629.

In [65]:
gamma=1.0
sigma=(0.5/100)
dt=1/264
r_ = (0.5/100)
r0 = curva(0)

In [66]:
numero_aleatorio=.1629

In [67]:
dt_gamma_r_ = dt * gamma * r_
sigma_sqdt = sigma * math.sqrt(dt)
result = [(0, r0),]
gamma_dt = gamma * dt
num_dias=1
pasos_dia=1
r = r0
for i in range(1, (num_dias + 1) * pasos_dia):
    r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * numero_aleatorio # Discretización de Euler
    result.append((i * dt, r))


In [68]:
df_sim = pd.DataFrame(result, columns=['t', 'tasa'])

In [70]:
df_sim

,t,tasa
0,0.000000,0.0008062107276586798
1,0.003788,0.000872225


### Pregunta 3

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 382 y 554 días.

In [35]:
#Se utiliza la libreria from scipy.interpolate import interp1d, para poder interpolar de manera lineal

crv= interp1d(df_curva['plazo'],df_curva['tasa'], 'linear' )

In [36]:
#Valores de días que se encuentran en el enunciado de la pregunta 
t1=382
t2= 554

#se utiliza la interpolación:
r1=crv(t1) 
r2=crv(t2)

#Cálculo de los factores de descuento:
df1=math.exp(-r1*t1 /365)
df2=math.exp(-r2*t2 /365)
fwd12 =(df1/df2 - 1.0) *360.0 /(t2-t1) #formula para obtener la tasa forward entre ambos días.


print (f'La tasa forward entre 382 y 554 días es igual a:  {fwd12:.8%}')

La tasa forward entre 382 y 554 días es igual a:  0.05032025%


### Pregunta 4

Considere un swap a 1Y con cupones semestrales de 1.0% (lineal). Si la tasa
cupón cero a 0.5Y es 0.9% (exp), calcule la tasa cero a 1Y.

In [71]:
cupones_sem=0.01
tasa_cupon_cero=(0.9/100)



### Pregunta 5

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,110.00 (365 días después).

In [42]:
ICP0 = 10000.00 
ICP365 = 10110.00
d=365

In [45]:
#TNA: tasa nominal anual 
TNA= round(((ICP365/ICP0) -1) *(360/d),4)

print (f'El valor de la tasa TNA es:  {TNA:.4%}')

El valor de la tasa TNA es:  1.0800%


### Pregunta 6

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.

Al valorizar un payoff con simulaciones de Montecarlo utilizando el modelo de Hull-White en diferentes medidas, se obtendrán **diferentes valores AD: ¿?**. La simulación de montecarlo utilizando el modelo de Hull.White en la medida libre de riesgo utiliza una función theta que depende del tiempo, y esta función se calibra  para que el modelo se ajuste perfectamente al valor de mercado de los bonos cero cupón. La forma de traer a valor presente este payoff es devolviendose por las trayectorias que generé, tengo el valor del instrumento en esta trayectoria y me lo debo traer a valor presente. Al traerlo a valor presente, debo traerme el producto de todos los factores de descuento que generé en la trayectoria y ahi obtengo un valor presente, luego hago otra trayectoria, calculo el valor y me lo traigo a valor presente. Una vez que se finaliza este proceso, debo sacar el promedio y eso será el valor del instrumento. Es importante rescatar cada una de las tasa que se obtienen en la simulación, ya que con ellas calcularemos los factores de descuento que se utilizan para descontar los cupones del bono. Estos son los pasos que se deben seguir cuando estamos utilizando la medida libre de riesgo.

En cambio, al realizarlo con la medida forward, es distinto, pues primero debo calcular una función theta en la medida forward, por lo que la función cambia en relacion con el otro caso. Se debe encontrar un nuevo drift que corresponde al ajusto que se debe realizar para encontrar el valor del payoff. Además el calculo del valor presente se realiza de otra forma, pues se utilza una tasa directamente de la curva cupon cero creada. 

### Pregunta 7

Ceteris paribus una opción sobre una activo con reversión a la media vale menos
que una opción sobre un activo sin reversión a la media : verdadero 

Las opciones sin reversión a la media son más caras que las opciones con reversión a la media mientras que todo el resto se mantenga constante. Esto queda demostrado con el siguiente ejemplo, si tenemos dos subyacentes, en donde su coeficiente de volatilidad es el mismo. Ambas opciones parten en el mismo punto inicial pero una opción tiene reversión a la  media y la otra no. La opción con reversión a la media va a mantenerse mucho más acotado, porque la varianza del subyacente se mantiene aproximadamente en el mismo nivel, en cambio, la varianza del instrumento sin reversión a la media podría explotar. Cuando un modelo no tiene reversión a la media, siempre ocurrirá el subyacente superará cierto nivel, por lo que esta opción valdrá más. En una opción sobre un activo con reversión a la media, existen niveles inalcanzables. 

Se deduce que en el largo plazo la distribución de valores posibles va a estar centrada en la tasa de largo plazo y tendrá la varianza de largo plazo y por lo tanto hay escenarios que son inalcanzables en el largo plazo. Por lo que las opciones van a tener más valor cuando el subyacente no tiene reversión a la media. 